In [2]:
!pip install opensearch-py

  Using cached https://files.pythonhosted.org/packages/1f/5e/e1a19eb99313da69e92095858bf828ea849242e3562de6ccf6b88866c378/opensearch_py-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d9/5a/e7c31adbe875f2abbb91bd84cf2dc52d792b5a01506781dbcf25c91daf11/six-1.16.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c96257acb4fce61e7de14/certifi-2024.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ec/57/56b9bcc3c9c6a792fcbaf139543cee77261f3651ca9da0c93f5c1221264b/python_dateutil-2.9.0.post0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/20/748e38b466e0819491f0ce6e90ebe4184966ee304fe483e2c414b0f4ef07/requests-2.32.2-py3-none-any.whl
  Using cached https://files.pythonhosted.

In [1]:
from opensearchpy import OpenSearch
host = 'vpc-aw-fdc-opensearch-qa-01-klbmt4ru7oksvsemnnmjpohfw4.us-east-1.es.amazonaws.com'
port = 443
auth = ('qa-developer', 'wF5341W8=~') # For testing only. Don't store credentials in code.
#ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

In [2]:
query = {
  "size" : 10000,
  "sort" : [
        { "@timestamp" : {"order" : "desc"}}
  ],
  "query": {
    "bool": {
        "must": [
            {"match_phrase" : {"kubernetes.labels.app": "fosfor-gatekeeper"}},
            {"regexp": {"message": "[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}"}}
        ],
        "filter" : { "bool" :   {   
            "must_not": [
                {"regexp" : {"message": "elb"}},
                {"regexp" : {"message": "gatewayconfig"}},
            ]
            }
        }
      }
    }
}
    
response = client.search(
    body = query,
    index = 'filebeat-qa-*'
)

#response

In [3]:
import sys

count=0
for hit in response['hits']['hits']:
    count=count+1
    print(hit["_source"]["message"])
    if count > 10:
        break


2024-05-24 07:47:41.646  INFO 1 --- [or-http-epoll-3] c.l.f.c.GatewayConfig                    : user id 5c6e27e4-b1a0-40e2-9684-a08effec431b
10.229.14.49 - - [24/May/2024:07:47:41 +0000] "POST /dag-backend/secured/api/v1/flow/saveFlow HTTP/1.1" 201 672 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36" "3.230.189.192"
10.229.12.227 - - [24/May/2024:07:47:41 +0000] "PUT /nlp-config/v2/datasetConfig?dataset_id=datasetId_Not%20Found&version=draft HTTP/1.1" 200 154 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36" "52.205.198.241"
2024-05-24 07:47:41.604  INFO 1 --- [or-http-epoll-1] c.l.f.c.GatewayConfig                    : user id 6c4c9411-4dfd-4eeb-b340-10b6b1560a95
2024-05-24 07:47:41.596  INFO 1 --- [or-http-epoll-4] c.l.f.c.GatewayConfig                    : user id 5c6e27e4-b1a0-40e2-9684-a08effec431b
10.229.15.173 - - [24/May/2024:07:47:41 +0000] 

In [4]:
import sys

l_arr = []
for hit in response['hits']['hits']:
    s_arr = []
    str = hit["_source"]["message"].split()
    if len(str) > 9:
        s_arr.append(str[6])
        s_arr.append(str[8])
        s_arr.append(str[9])
        l_arr.append(s_arr)

import pandas as pd
df = pd.DataFrame(l_arr, columns=['URL', 'CODE', 'TIME'])

df["TIME"] = pd.to_numeric(df['TIME'], errors='coerce')
df.dropna(subset=['TIME'], inplace=True)
df.sort_values(by='TIME', inplace=True, ascending=False)

<ipython-input-4-6cc623a36ce9>:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
df[0:110].head(50)

,URL,CODE,TIME
959,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
2510,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
9535,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
3513,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
8526,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
1229,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
907,/fdc/444.c7f38b428d52b332.bundle.js,200,4926804.0
665,/job/755.1ab44045d7c372c6.bundle.js,200,4477707.0
8329,/pipeline/src_lib_containers_flowdesign_index_...,200,3535626.0
8330,/pipeline/vendors-node_modules_material-ui_cor...,200,3530627.0


In [7]:
df2 = df[df["TIME"] > 60000]

In [8]:
df2["URL"].unique()

array(['/monitor-pipeline/vendors-node_modules_fosfor-design-system-react_dist_index_js-node_modules_moment_locale_af_j-0e93ea.a7758a25f345f21f.bundle.js',
       '/pipeline/vendors-node_modules_date-io_date-fns_build_index_esm_js-node_modules_material-ui_core_esm_Bo-f30ba0.74625b7a8bbafcf8.bundle.js',
       '/fdc/578.515f3538683840c3.bundle.js',
       '/monitor-pipeline/vendors-node_modules_material-ui_core_esm_Checkbox_Checkbox_js-node_modules_material-ui_core_-97e514.d4bc7325bf2ea23f.bundle.js',
       '/job/755.1ab44045d7c372c6.bundle.js',
       '/pipeline/src_lib_containers_flowdesign_index_js.663365b4230432da.bundle.js',
       '/pipeline/vendors-node_modules_material-ui_core_colors_green_js-node_modules_material-ui_icons_Add_js-n-d1df88.4aaba4de31d72943.bundle.js',
       '/templates/82e5205d5c69794eb1dd5a34e4aef086/38890189-77f6-43c9-960b-3df992c9df29/Python-3.9-f4b28591-d3cc-40d9-bbd1-d0d1539a6cdd/static/nbclassic/notebook/js/main.min.js?v=ec0e2a68f0d65bebacb5e8fb0d53085458

In [16]:
df_fail = df[df["CODE"] == "500"]
df_fail

,URL,CODE,TIME
9792,/dag-backend/secured/api/v1/flow/getPublishedF...,500,7594.0
3580,/dag-backend/secured/api/v1/flow/saveFlow,500,275.0
9590,/refract/mlflow/ajax-api/2.0/mlflow/experiment...,500,265.0
6742,/refract/mlflow/ajax-api/2.0/mlflow/experiment...,500,265.0
1909,/refract/mlflow/ajax-api/2.0/mlflow/experiment...,500,265.0
8519,/project-manager/mosaic-console-backend/secure...,500,251.0
3258,/project-manager/mosaic-console-backend/secure...,500,243.0
3245,/project-manager/connections/api/Scan/v1/preview,500,232.0
5356,/dag-backend/secured/api/v1/validation/create,500,196.0
7371,/notebooks/api/v1/git-repo,500,169.0
